In [17]:
import os
import sys
import json
from glob import glob
from google.colab import drive
from google.colab import files

ROOT = "/content/drive"

try:
    drive.mount(ROOT)
except:
    drive.mount(ROOT, force_remount=True)

TASK_PATH = "MyDrive/Task/plastic-segmentation/Data"
# gdrive_url = "https://drive.google.com/drive/folders/1AjbGB_cl0XUoBF9ezKZ-4ilIer5qdyKh?usp=sharing"
# gdrive 활용하는 것은 다음 번에 시도하기

TRN_PATH = os.path.join(ROOT, TASK_PATH, "Train")
TST_PATH = os.path.join(ROOT, TASK_PATH, "Test")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
PCLS = ["PE", "PET", "PP"]  # , "PS" 테스트 데이터가 이상하게 들어가서 우선은 리스트에 넣지 않고 빼도록 함

trn_paths = {"image_path" : [], "annot_path": []}
tst_paths = {"image_path" : [], "annot_path": []}

for fpath in [TRN_PATH, TST_PATH]:
    for pcls in PCLS:
        img_path = os.path.join(fpath, pcls)
        img_paths = sorted(glob(img_path + "/*.jpg"))
        if fpath.split('/')[-1]=="Train":
            annot_path = os.path.join(fpath, "annotation", pcls)
            trn_paths["image_path"].extend(img_paths)
            trn_paths["annot_path"].extend(sorted(glob(annot_path + "/*.json")))
        elif fpath.split('/')[-1]=="Test":
            annot_path = os.path.join(fpath, "annotations", pcls)
            tst_paths["image_path"].extend(img_paths)
            tst_paths["annot_path"].extend(sorted(glob(annot_path + "/*.json")))

In [63]:
import pandas as pd
pd.DataFrame(trn_paths).iloc[0,:].apply(lambda x:x.split('/')[-1])

image_path     PE_074_10.jpg
annot_path    PE_074_10.json
Name: 0, dtype: object

In [70]:
# trn_paths["datas"] = list(map(lambda x: (x.split('/')[-4]).lower(),trn_paths["annot_path"]))
# tst_paths["datas"] = list(map(lambda x: (x.split('/')[-4]).lower(),tst_paths["annot_path"]))
## 이미 변수로 구분해서 필요없음

trn_paths["classes"] = list(map(lambda x: (x.split('/')[-1]).split('_')[0],trn_paths["annot_path"]))
trn_paths["feats_1"] = list(map(lambda x: (x.split('/')[-1]).split('_')[1],trn_paths["annot_path"]))
trn_paths["feats_2"] = list(map(lambda x: (x.split('/')[-1]).split('_')[-1][:-5],trn_paths["annot_path"]))

tst_paths["classes"] = list(map(lambda x: (x.split('/')[-1]).split('_')[0],tst_paths["annot_path"]))
tst_paths["feats_1"] = list(map(lambda x: (x.split('/')[-1]).split('_')[1],tst_paths["annot_path"]))
tst_paths["feats_2"] = list(map(lambda x: (x.split('/')[-1]).split('_')[-1][:-5],tst_paths["annot_path"]))

pd.DataFrame(trn_paths)

,image_path,annot_path,classes,feats_1,feats_2
0,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PE,074,10
1,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PE,074,100
2,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PE,074,101
3,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PE,074,102
4,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PE,074,103
...,...,...,...,...,...
2995,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PP,058,8929
2996,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PP,058,8995
2997,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PP,058,908
2998,/content/drive/MyDrive/Task/plastic-segmentati...,/content/drive/MyDrive/Task/plastic-segmentati...,PP,058,910


In [23]:
# trn_annots = 
# for apath in trn_paths["annot_path"][0]:
with open(trn_paths["annot_path"][0], "r") as jsf:
    jsd = json.loads(jsf.read())
print(jsd)

{'images': [{'width': 2048, 'height': 2048, 'id': 1, 'file_name': 'PE_074_10.jpg'}], 'categories': [{'id': 1, 'name': 'pet'}, {'id': 2, 'name': 'ps'}, {'id': 3, 'name': 'pp'}, {'id': 4, 'name': 'pe'}], 'annotations': [{'id': 1, 'image_id': 1, 'category_id': 4, 'metainfo_id': 74, 'segmentation': [[890, 837, 883, 841, 877, 847, 872, 854, 868, 861, 863, 868, 856, 875, 851, 881, 845, 888, 842, 895, 837, 902, 834, 909, 830, 916, 826, 923, 822, 930, 818, 937, 814, 944, 809, 951, 807, 958, 802, 965, 800, 972, 795, 979, 793, 986, 792, 993, 792, 1000, 795, 1007, 802, 1011, 808, 1017, 815, 1019, 822, 1024, 829, 1028, 836, 1033, 843, 1036, 850, 1041, 857, 1044, 864, 1049, 871, 1052, 878, 1056, 885, 1058, 892, 1058, 899, 1054, 906, 1049, 909, 1042, 914, 1035, 918, 1028, 922, 1021, 924, 1014, 930, 1007, 934, 1000, 937, 993, 941, 987, 945, 980, 948, 973, 952, 966, 957, 959, 960, 952, 964, 945, 966, 938, 966, 931, 967, 924, 968, 917, 970, 910, 974, 903, 980, 896, 983, 889, 982, 882, 977, 876, 970, 87

In [24]:
jsd.keys()

dict_keys(['images', 'categories', 'annotations', 'info', 'metainfo'])

In [26]:
jsd["images"][0].keys()

dict_keys(['width', 'height', 'id', 'file_name'])

In [62]:
from collections import OrderedDict, defaultdict
# OrderedDict(jsd["categories"])
# tuple(list(map(lambda x : tuple(x), jsd["categories"])))
# list(map(lambda x : tuple(x.items()), jsd["categories"]))
# list(map(lambda x : tuple(x.values()), jsd["categories"]))

## 여기부터 고쳐야함
cats = defaultdict(list)
for cts in jsd["categories"]:
    cats.update(cts)
print(cats)

defaultdict(<class 'list'>, {'id': 4, 'name': 'pe'})
